In [2]:
import os
import sqlite3


In [4]:
# generate_sample_trades.csv
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os

# Ensure the data folder exists

# Parameters
n_trades = 100
symbols = ['AAPL', 'MSFT', 'TSLA', 'GOOG', 'AMZN', 'NVDA', 'META']
brokers = ['Goldman', 'Morgan Stanley', 'JP Morgan', 'Barclays', 'Citi']
sectors = ['Technology', 'Consumer', 'Financials', 'Healthcare']

# Generate random trades
trades = []
start_datetime = datetime(2025, 12, 5, 9, 30)  # trading day start

for i in range(n_trades):
    order_id = f"O{i+1:04d}"
    symbol = random.choice(symbols)
    side = random.choice(['BUY', 'SELL'])
    order_qty = random.randint(10, 1000)
    executed_qty = int(order_qty * random.uniform(0.8, 1.0))
    price_base = random.uniform(50, 500)
    arrival_price = round(price_base * random.uniform(0.995, 1.005), 2)
    execution_price = round(arrival_price * random.uniform(0.995, 1.005), 2)
    decision_price = round(price_base * random.uniform(0.995, 1.005), 2)
    vwap = round(price_base * random.uniform(0.995, 1.005), 2)
    start_time = start_datetime + timedelta(seconds=random.randint(0, 3600))
    end_time = start_time + timedelta(seconds=random.randint(60, 600))
    broker = random.choice(brokers)
    sector = random.choice(sectors)
    market_move_pct = np.random.uniform(-0.005, 0.005)

    trades.append([
        order_id, symbol, side, order_qty, executed_qty,
        execution_price, arrival_price, decision_price, vwap,
        start_time.strftime("%Y-%m-%d %H:%M:%S"),
        end_time.strftime("%Y-%m-%d %H:%M:%S"),
        broker, sector, market_move_pct
    ])

# Create DataFrame
columns = [
    "order_id", "symbol", "side", "order_qty", "executed_qty",
    "execution_price", "arrival_price", "decision_price", "vwap",
    "start_time", "end_time", "broker", "sector", 'market_move_pct'
]

df = pd.DataFrame(trades, columns=columns)

# Save CSV
# df.to_csv("data/sample_trades.csv", index=False)
# print("sample_trades.csv created in data/ folder")


In [5]:
data_path = os.path.join("..", "data", "sample_trades.csv")

In [6]:
df.to_csv(data_path, index=False)

In [15]:
# db_path = os.path.join("..", "database", "tca.db")
# conn = sqlite3.connect(db_path)
# conn = sqlite3.connect(db_path)

# cursor = conn.cursor()

# cursor.execute("""
# CREATE TABLE IF NOT EXISTS trades (
#     order_id TEXT PRIMARY KEY,
#     symbol TEXT NOT NULL,
#     side TEXT CHECK(side IN ('BUY','SELL')) NOT NULL,
#     order_qty INTEGER NOT NULL,
#     executed_qty INTEGER NOT NULL,
#     execution_price REAL NOT NULL,
#     arrival_price REAL NOT NULL,
#     decision_price REAL,
#     vwap REAL,
#     start_time TEXT NOT NULL,
#     end_time TEXT NOT NULL,
#     broker TEXT,
#     sector TEXT
# )
# """)

# # Commit changes and close
# conn.commit()
# conn.close()

# print("tca.db created with table 'trades'")


tca.db created with table 'trades'


In [17]:

# csv_path = "../data/sample_trades.csv"
# db_path = "../database/tca.db"

# df_csv = pd.read_csv(csv_path)
# print(len(df_csv))

# with sqlite3.connect(db_path) as conn:
#     df_csv.to_sql("trades", conn, if_exists="append", index=False)
    
# with sqlite3.connect(db_path) as conn:
#     df_sql = pd.read_sql_query("SELECT * FROM trades LIMIT 5;", conn)

# print(df_sql)


100
  order_id symbol  side  order_qty  executed_qty  execution_price  \
0    O0001   MSFT  SELL        218           215           191.71   
1    O0002   NVDA   BUY        322           312            58.82   
2    O0003   AAPL   BUY        545           466           193.90   
3    O0004   GOOG  SELL         26            23           144.58   
4    O0005   MSFT  SELL        861           831           250.27   

   arrival_price  decision_price    vwap           start_time  \
0         191.09          191.38  191.59  2025-12-05 09:43:33   
1          58.60           58.54   58.97  2025-12-05 09:31:01   
2         193.81          193.50  195.11  2025-12-05 10:14:29   
3         144.11          144.83  144.53  2025-12-05 09:36:44   
4         250.38          248.99  250.47  2025-12-05 09:51:35   

              end_time          broker      sector  
0  2025-12-05 09:51:01         Goldman  Healthcare  
1  2025-12-05 09:40:57  Morgan Stanley  Technology  
2  2025-12-05 10:16:53         

In [7]:
import random

db_path = os.path.join("..", "database", "tca.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# --- Add column if it doesn't exist --- #
cursor.execute("""
ALTER TABLE trades
ADD COLUMN market_move_pct REAL
""")

# --- Generate random market move percentages for all trades --- #
cursor.execute("SELECT order_id, decision_price FROM trades")
rows = cursor.fetchall()

for order_id, decision_price in rows:
    # Random market move in ±0.5%
    market_move_pct = round(random.uniform(-0.005, 0.005), 6)  # fractional
    cursor.execute(
        "UPDATE trades SET market_move_pct = ? WHERE order_id = ?",
        (market_move_pct, order_id)
    )

# Commit and close
conn.commit()
conn.close()

print("market_move_pct column added and populated with random ±0.5% values")


market_move_pct column added and populated with random ±0.5% values


In [9]:
with sqlite3.connect(db_path) as conn:
    df_sql = pd.read_sql_query("SELECT symbol, market_move_pct FROM trades LIMIT 5;", conn)

print(df_sql)

  symbol  market_move_pct
0   MSFT        -0.001017
1   NVDA         0.000445
2   AAPL         0.004004
3   GOOG         0.004364
4   MSFT        -0.000742
